In [1]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from config import *

/home/azureuser/Ambarish/sentence-llama-index/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient(URL,port=6333)

In [3]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

In [4]:
from llama_index.llms.azure_openai import AzureOpenAI
llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name=deployment_id_gpt4,
    api_key=key,
    azure_endpoint=endpoint,
    api_version=api_version,
)

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [6]:
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2",max_length=512
)

In [7]:
from llama_index.core import Settings
from llama_index.core  import VectorStoreIndex

Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [9]:
query_engine = index.as_query_engine()

In [20]:
#query = "What is  ENERGY INTENSITY AT TSJ?"
#query = "Describe about Tata Steel Kalinganagar?"
query = "What should the government do about road safety?"
query = "What is the Golden Quadrilateral ?"

In [21]:
response = query_engine.query(query)

In [22]:
print(response.get_formatted_sources())
print("query was:", query)
print("answer was:", response)

> Source (Doc id: be274025-6341-403c-a37c-14ebf15879ec): —Widening of about 6,500km of National 
Highways to 6-lane on Design-Build-
Finance-Operate (DBFO...

> Source (Doc id: f63a72f1-0733-482f-b802-6ba44e3353f7): The assets created are being 
operated and maintained through 
private agencies.National Highways...

> Source (Doc id: 8e598763-fdf0-4d85-bf92-a3444a7d05a8): —One of the most attractive destinations for Foreign Direct Investment (FDI). 

> Source (Doc id: c70e6b99-43a2-4b2b-a748-079f90865086): —Strengthening and widening of another 
20,000 km of National Highways to two 
lane with paved sh...

> Source (Doc id: 96e76285-5231-486c-8086-92395d3dcaf6): The Government of India launched 
the NHDP  in two phases in 1999 at an 
estimated cost of almost...
query was: What is the Golden Quadrilateral ?
answer was: The Golden Quadrilateral is a component of the National Highways Development Project (NHDP) in India, which connects Delhi, Mumbai, Chennai, Kolkata, and Delhi, covering 

In [23]:
from pprint import pprint

In [24]:
pprint(str(response))

('The Golden Quadrilateral is a component of the National Highways Development '
 'Project (NHDP) in India, which connects Delhi, Mumbai, Chennai, Kolkata, and '
 'Delhi, covering a total length of 5,846 km.')


In [25]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

In [26]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
    llm = llm,
)

In [27]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

In [28]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query(query)

In [29]:
pprint(str(response))

('The Golden Quadrilateral is a component of the National Highways Development '
 'Project (NHDP) in India, which spans a total length of 5,846 km and connects '
 'Delhi, Mumbai, Chennai, Kolkata, and Delhi.')
